# Multi-process models in pywatershed: PRMS-legacy instantiation

Because `pywatershed` has its primary legacy in the Precipitation-Runoff Modeling System (PRMS, Regan et al., 2015), `pywatershed` supports PRMS-model instantation from legacy PRMS input files. THe traditional input files are the `control` file, the parameter file, and the CBH files. While the CBH files need to be pre-processed to NetCDF format, PRMS control and parameter files as supported. This changes the way the model is instantiated, as we will see below.

**Show pre-processing of CBH files in another notebook***

As in the previous notebooks, we'll look at running PRMS processes moels on the Delaware River Basin (DRB) subdomain of the NHM for a 2 year period using `pywatershed`.

## Prerequisites


In [ ]:
# auto-format the code in this notebook
%load_ext jupyter_black

In [ ]:
import pathlib as pl
from platform import processor
from pprint import pprint
from sys import platform

import pydoc

import hvplot.pandas  # noqa
import numpy as np
import pywatershed as pws
import xarray as xr

from helpers import gis_files

gis_files.download()  # this downloads GIS files

The domain directory is where we have all the required inputs to run this model (among others) and `nb_output_dir` is where this notebook will write its output. 

In [ ]:
domain_dir = pws.constants.__pywatershed_root__ / "data/drb_2yr"
nb_output_dir = pl.Path("./02_prms_legacy_models")

## An NHM multi-process model: PRMS-legacy instantation

The 8 conceptual `Process` classes that comprise the NHM are, in order:

In [ ]:
nhm_processes = [
    pws.PRMSSolarGeometry,
    pws.PRMSAtmosphere,
    pws.PRMSCanopy,
    pws.PRMSSnow,
    pws.PRMSRunoff,
    pws.PRMSSoilzone,
    pws.PRMSGroundwater,
    pws.PRMSChannel,
]

A multi-process model, comprised of the above `Processes` (see notebook 00), is assembled by the `Model` class. We can take a quick look at the first 22 lines of help on `Model`:

In [ ]:
# this is equivalent to help() but we get the multiline string and just look at part of it
model_help = pydoc.render_doc(pws.Model, "Help on %s")
# the first 22 lines of help(pws.Model)
print("\n".join(model_help.splitlines()[0:22]))

The `help()` mentions that there are 2 distinct ways of instantiating a `Model` class. In this notebook, we focus on the PRMS-legacy instantiation (see previous notebook for the pywatershed-centric way).

With the PRMS-legacy approach, the first argument is a "process list", which is what we defined with the 8 NHM classes above. In addition, we'll supply the `control` and `parameters` arguments. The full `help()` describes PRMS-legacy instantation and provides examples. Please use it for reference and more details. Here we'll give an extended concrete example. 

We load a `PrmsParameters` object from a PRMS-native file. We'll check that it is an instance/subclass of the `Parameters` class. Then we'll return the object (which invokes the `__repr__` method on the object, just giving information about it) to see what we got.

In [ ]:
params = pws.parameters.PrmsParameters.load(domain_dir / "myparam.param")
print(isinstance(params, pws.Parameters))
params

We got a `PrmsParameters` object which is a subclass of `Parameters`. We now load control file into a `Control` object, 

In [ ]:
control = pws.Control.load(domain_dir / "control.test")
control

Now we'll edit this control object. First we'll reduce the total simulation time to six months for the purposes of this demonstration (but feel free to increase this to the full 2 years available, if you like). Next we'll specify several global options, including the location of the atmospheric forcing/input data, the budget type, and the calculation method.

In [ ]:
control.edit_end_time(np.datetime64("1979-07-01T00:00:00"))
control.options = control.options | {
    "input_dir": domain_dir,
    "budget_type": "warn",
    "calc_method": "numba",
    "netcdf_output_dir": nb_output_dir / "nhm",
}

Now we can initialize the NHM model.

In [ ]:
nhm = pws.Model(
    nhm_processes,
    control=control,
    parameters=params,
)

Numba is a Python package for just-in-time compiling. It takes code written using the `numpy` package and accelerates it by compiling at run time. The processes listed above compiled their code on initialization of the model. The processes that benefit from jit compiling are challenging to vectorize and have a loop over space which is accelerated by the compiling. The remaining processes, not jit compiled, are all vectorized (in space and time): PRMSSolarGeometry, PRMSAtmosphere. 

Now we can run the model, requesting it finalize at the end, and we'll time the result.

In [ ]:
%%time
nhm.run(finalize=True)

Now that we've run our NHM model on the DRB, let's take a look at the simulated streamflow. Still in memory are the streamflow values from the final timestep, we'll plot those on the stream network overlaid on the watershed. 

In [ ]:
proc_plot = pws.analysis.ProcessPlot(gis_files.gis_dir / "drb_2yr")
proc_name = "PRMSChannel"
var_name = "seg_outflow"
proc = nhm.processes[proc_name]
display(proc_plot.plot(var_name, proc))

Above we see the spatial domain, the outline of its extent, the network on which streamflow is calculated, and the final simulated values of streamflow. 

Let us turn towards the model structure in more detail: how do atmospheric inputs/forcings result in the simulated streamflow above? We will produce the model graph which shows the flow of information from the input files through all the process representations, all the way down to the channel streamflow. First we print a color legend for each represented process in the NHM. Each process is outlined by a box of this color and values/fluxes flowing from a process have the color of the originating process. Finally, a variable outlined (above and on the sides) participates in the mass budget of its process. This diagram gives some very specific information of the model conceptualization, how the processes relate to each other, and the complexity of the indivdual processes. (Note that the underlying graphviz/dot program that generates the plot is not fully working on Mac ARM/M1, so plots here and below are less detailed if you are are using such a machine, the notebooks in the repo will be complete for your reference.) Each process's data is placed into one of three categories: inputs(blue), parameters(orange), and variables(green). All of this information is public for each process (indeed in static methods) so we can produce these plots programatically without needing to run the Model. The Model object contains all the information needed to generate the plot when it is initialized.

In [ ]:
palette = pws.analysis.utils.colorbrewer.nhm_process_colors(nhm)
pws.analysis.utils.colorbrewer.jupyter_palette(palette)
show_params = not (platform == "darwin" and processor() == "arm")
try:
    pws.analysis.ModelGraph(
        nhm,
        hide_variables=False,
        process_colors=palette,
        show_params=show_params,
    ).SVG(verbose=True, dpi=48)
except:
    print("In some cases, dot fails on Mac ARM machines.")

## NHM Submodel for the Delaware River Basin 
Suppose you wanted to change parameters or model process representation in the PRMSSoilzone to better predict streamflow. As the model is 1-way coupled, you can simply run a submodel starting with PRMSSoilzone and running through PRMSChannel.

In [ ]:
submodel_processes = [pws.PRMSSoilzone, pws.PRMSGroundwater, pws.PRMSChannel]

In [ ]:
# params = pws.parameters.PrmsParameters.load(domain_dir / "myparam.param")
control = pws.Control.load(domain_dir / "control.test")
control.edit_n_time_steps(90)
control.options = control.options | {
    "input_dir": nb_output_dir / "nhm",
    "budget_type": "warn",
    "calc_method": "numba",
    "netcdf_output_dir": nb_output_dir / "submodel",
}

In [ ]:
submodel = pws.Model(
    submodel_processes,
    control=control,
    parameters=params,
)

pws.analysis.ModelGraph(
    submodel,
    hide_variables=not show_params,
    show_params=show_params,
    process_colors=palette,
).SVG(verbose=True, dpi=48)

Now we can initalize output and run the submodel.

In [ ]:
%%time
submodel.run(finalize=True)